In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse

In [ ]:
# Initial global variables
keywords = ['food']
locations = ['San Francisco, CA']
page = 4

# Regex pattern for phone
pattern_phone = r'\(\d{3}\)\s\d{3}-\d{4}'

# Looping logic to iterate through every location and keyword
for kl in locations:
    key_location = urllib.parse.quote(kl)
    for kw in keywords:
        key_word = kw.lower().strip().replace(' ', '+')

        # Individual dict for every keyword per each location
        df = {'Name': [], 'Website': [], 'Phone': [], 'Street': [], 'Address': []}

        # Handle exceptions in case of lack of data
        try:
            for pg in range(1, page + 1):
                url = f"https://www.yelp.com/search?find_desc={key_word}&find_loc={key_location}&start={(pg - 1) * 10}"
                source = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
                soup = BeautifulSoup(source.text, 'html.parser')
                businesses = soup.find_all('div', class_='container__09f24__21w3G')

                for business in businesses:
                    # Variable declaration
                    name, website, phone, street, locality = '', '', '', '', ''
                    address = street + ', ' + locality

                    # Variable to store name data
                    name_elem = business.find('a', class_='link-color--inherit__09f24__3PYlA')
                    if name_elem:
                        name = name_elem.get_text(strip=True)

                    # Variable to store website
                    website_elem = business.find('a', class_='css-166la90')
                    if website_elem:
                        website = website_elem['href']

                    # Variable to store phone number
                    phone_elem = business.find('p', class_='css-1p9ibgf')
                    if phone_elem:
                        phone_match = re.search(pattern_phone, phone_elem.get_text())
                        if phone_match:
                            phone = phone_match.group()

                    # Variable to store address
                    address_elem = business.find('p', class_=' css-qyp8bo')
                    if address_elem:
                        address_parts = address_elem.get_text().split('\n')
                        if len(address_parts) > 1:
                            street = address_parts[1]
                        if len(address_parts) > 2:
                            locality = address_parts[2]

                    # Concatenate street and locality
                    address = street + ', ' + locality

                    # Push data into dictionary
                    if locality:
                        df['Name'].append(name)
                        df['Website'].append(website)
                        df['Phone'].append(phone)
                        df['Street'].append(street)
                        df['Address'].append(address)

        except Exception as e:
            print(f'Partial data extracted for {kw} in {kl}. Error: {e}')
        else:
            print(f'Complete data extracted for {kw} in {kl}')

        # Change the dict to DataFrame
        dataframe = pd.DataFrame(df)

        # Download dataset as CSV
        k_word = kw.replace('+', '_')
        k_location = kl.replace(",", "").replace(" ", "_")
        dataframe.to_csv(f'{k_word}_{k_location}.csv')

Complete data extracted for education in Toronto, ON
